In [8]:
import sys
sys.path.append('../..')
import pybeamtools

In [9]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
def time():
    return time.time()*0.5

t = 0.0
def fixed_time():
    return t

In [17]:
from pybeamtools.sim.pddevices import EchoDevice, EchoDeviceOptions, TRIGSPEC
echo = EchoDevice(options=EchoDeviceOptions(name='echo', data={'TEST:ECHO:1':5}))
echo2 = EchoDevice(options=EchoDeviceOptions(name='echo2', data={'TEST:ECHO:2':15}))

In [18]:
echo.options.dict()

{'name': 'echo', 'device_type': 'echo', 'data': {'TEST:ECHO:1': 5}}

In [ ]:
eo = EchoDeviceOptions.parse_obj(echo.options.dict())

In [19]:
from pybeamtools.sim.pddevices import ProxyDevice, ProxyDeviceOptions
proxy = ProxyDevice(ProxyDeviceOptions(name='proxy', channel_map={'TEST:PROXY:1':{'TEST:ECHO:1': TRIGSPEC.PROPAGATE}}))
proxy1b = ProxyDevice(ProxyDeviceOptions(name='proxy1b', channel_map={'TEST:PROXY:1B':{'TEST:ECHO:2': TRIGSPEC.IGNORE}}))
proxy2 = ProxyDevice(ProxyDeviceOptions(name='proxy2', channel_map={'TEST:PROXY:2':{'TEST:PROXY:1': TRIGSPEC.PROPAGATE}}))
proxy2b = ProxyDevice(ProxyDeviceOptions(name='proxy2b', channel_map={'TEST:PROXY:2B':{'TEST:PROXY:1': TRIGSPEC.PROPAGATE}}))

In [20]:
from pybeamtools.sim.core import SimulationEngine, SignalEngineOptions
sim = SimulationEngine(SignalEngineOptions(time_function=fixed_time))
sim.TRACE = True

In [21]:
sim.options.dict()

{'history_length': 1000,
 'time_function': <function __main__.fixed_time()>,
 'devices': [],
 'periods': {}}

In [22]:
opt = SignalEngineOptions.parse_obj(sim.options.dict())

In [ ]:
sim.add_device(echo, period=3.0)
sim._update_device('echo')
sim.add_device(echo2, period=5.0)
sim._update_device('echo2')

In [ ]:
print(sim.channels_map_chain)
print(sim.channels_dep_chain)
print(sim.next_periodic_read_time)

In [ ]:
proxy = ProxyEngineDevice(name='proxy', channel_map={'TEST:PROXY:1':{'TEST:ECHO:1': TRIGSPEC.PROPAGATE}})
proxy1b = ProxyEngineDevice(name='proxy1b', channel_map={'TEST:PROXY:1B':{'TEST:ECHO:2': TRIGSPEC.IGNORE}})
proxy2 = ProxyEngineDevice(name='proxy2', channel_map={'TEST:PROXY:2':{'TEST:PROXY:1': TRIGSPEC.PROPAGATE}})
proxy2b = ProxyEngineDevice(name='proxy2b', channel_map={'TEST:PROXY:2B':{'TEST:PROXY:1': TRIGSPEC.PROPAGATE}})

In [ ]:
sim.add_device(proxy, period=6.0)
sim._update_device('proxy')
sim.add_device(proxy1b, period=7.0)
sim._update_device('proxy1b')
sim.add_device(proxy2, period=9.0)
sim._update_device('proxy2')
sim.add_device(proxy2b, period=11.0)
sim._update_device('proxy2b')

In [ ]:
print(sim.channels_map_chain)
print(sim.channels_dep_chain)
print(sim.next_periodic_read_time)

In [ ]:
#t = sim.time()

In [ ]:
t += 10
sim._time_step(t)

In [ ]:
sim.time()

In [ ]:
def f(sub, v):
    print(f'Channel input_var_change_callback', sub.name, v)
sub = sim.subscribe_channel('TEST:ECHO:1')
sub.add_callback(f)

In [ ]:
t += 10
sim._time_step(t)

In [ ]:
def f(sub, v):
    print(f'Channel input_var_change_callback 2', sub.name, v)
sub = sim.subscribe_channel('TEST:PROXY:2B')
sub.add_callback(f)

In [ ]:
t += 10
sim._time_step(t)

In [ ]:
from pybeamtools.sim.devices import MagnetEngineDevice
mag1 = MagnetEngineDevice('magneto','MAG_VAL','MAG_VAL_EXACT', initial_value=0, low=-2, high=2, noise=0.1, resolution=None, model='instant')

In [ ]:
sim.add_device(mag1, period=5.0)
sim._update_device('magneto')

In [ ]:
t += 1
sim._time_step(t)

In [ ]:
print(sim.channels_map_chain)
print(sim.channels_dep_chain)
print(sim.next_periodic_read_time)

In [ ]:
t += 1
sim._time_step(t)

In [ ]:
from pybeamtools.sim.devices import RPNEngineDevice
rpn1 = RPNEngineDevice(name='rpn1', output_name='TEST:RPN', rpn_expression='TEST:PROXY:2B 10 +')

In [ ]:
sim.add_device(rpn1, period=1.0)
sim._update_device(rpn1)

In [ ]:
t += 1
sim._time_step(t)

In [ ]:
def f(sub, v):
    print(f'Channel input_var_change_callback rpn', sub.name, v)
sub = sim.subscribe_channel('TEST:RPN')
sub.add_callback(f)

In [ ]:
t += 1
sim._time_step(t)

In [ ]:
from pybeamtools.sim.devices import Oscillator, ModelEngineDevice
dev_osc = Oscillator(name='osc1', period=60.0, amplitude=1.0, now=t)
mdev_osc = ModelEngineDevice(device=dev_osc)

In [ ]:
mdev_osc.channel_map

In [ ]:
sim.add_device(mdev_osc, period=1.0)
sim._update_device(mdev_osc)

In [ ]:
t += 1
sim._time_step(t)

In [ ]:
sim.latest_data

In [ ]:
sim.latest_data_timestamp

In [ ]:
sim.start_update_thread()
print()

In [ ]:
sim.stop_update_thread()